In [3]:
import json
import os
from pathlib import Path

# Global variables

In [4]:
TARGET_LANG_CORPORA = ['BAS19_ES', 'FOR19_PT', 'HAS21_HI', 'OUS19_AR', 'SAN20_IT']
TARGET_CORPUS_SIZES = [0, 20, 200, 2000]
LANG_TO_TEST_PATHS = {
    'ES': ['processed/BAS19_ES/BAS19_ES_test_2000.jsonl', 'processed/MHC/MHC_ES_test_3745.jsonl'],
    'PT': ['processed/FOR19_PT/FOR19_PT_test_2000.jsonl', 'processed/MHC/MHC_PT_test_3691.jsonl'],
    'HI': ['processed/HAS21_HI/HAS21_HI_test_500.jsonl', 'processed/MHC/MHC_HI_test_3565.jsonl'],
    'AR': ['processed/OUS19_AR/OUS19_AR_test_1000.jsonl', 'processed/MHC/MHC_AR_test_3570.jsonl'],
    'IT': ['processed/SAN20_IT/SAN20_IT_test_2000.jsonl', 'processed/MHC/MHC_IT_test_3690.jsonl']
}
BASH_SCRIPT_HEADER = """#!/bin/bash

json=$(cat paths.json)
configs_dir=$(echo "$json" | grep -o '"configs_dir": "[^"]*"' | cut -d'"' -f4)
results_dir=$(echo "$json" | grep -o '"output_dir": "[^"]*"' | cut -d'"' -f4)
data_dir=$(echo "$json" | grep -o '"data_dir": "[^"]*"' | cut -d'"' -f4)
"""

# Generate Monolingual Configs

In [6]:
TARGET_LANG_MODELS = ['pysentimiento/robertuito-base-uncased', 'neuralmind/bert-base-portuguese-cased', 'neuralspace-reverie/indic-transformers-hi-bert', 'aubmindlab/bert-base-arabertv02', 'Musixmatch/umberto-commoncrawl-cased-v1']
PATH_OUT_TEMPLATE = 'monoling_target_lang/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = 'monoling_target_lang/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = 'monoling_target_lang/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
    },
    'prediction_pipeline': {
        'no_nli': True,
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcherNoNLI': {
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}


In [7]:
bash_script = open('../scripts/run_eval_monoling_target_lang.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)
for target_lang_corpus, target_lang_model in zip(TARGET_LANG_CORPORA, TARGET_LANG_MODELS):
    bash_script.write(f'# ------ {target_lang_corpus} - {target_lang_model} ------\n')
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                config['predictor']['model'] = target_lang_model
                if size == 0:
                    config['predictor']['checkpoint'] = None
                else:
                    config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                path_out_dir = f'../configs/monoling_target_lang/{target_lang_corpus}/examples_{size}/RUN{seed}/'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate Configs for X

In [3]:
PATH_OUT_TEMPLATE = 'X/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = 'X/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = 'X/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
    },
    'prediction_pipeline': {
        'no_nli': True,
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcherNoNLI': {
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}

In [8]:
bash_script = open('../scripts/run_eval_X.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)
for target_lang_corpus in TARGET_LANG_CORPORA:
    bash_script.write(f'# ------ {target_lang_corpus} ------\n')
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                config['predictor']['model'] = 'cardiffnlp/twitter-xlm-roberta-base'
                if size == 0:
                    config['predictor']['checkpoint'] = None
                else:
                    config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                path_out_dir = f'../configs/X/{target_lang_corpus}/examples_{size}/RUN{seed}/'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate Eval Configs for Non-NLI Models 

In [3]:
MODEL_NAME = 'cardiffnlp/twitter-xlm-roberta-base'
ENGLISH_DATASETS = ['X_DEN', 'X_FEN', 'X_KEN']
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
    },
    'prediction_pipeline': {
        'no_nli': True,
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcherNoNLI': {
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}

In [4]:
bash_script = open('../scripts/run_eval_bin.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)
for eng_dataset in ENGLISH_DATASETS:
    bash_script.write(f'# -------- {eng_dataset} --------\n')
    for target_lang_corpus in TARGET_LANG_CORPORA:
        bash_script.write(f'# ------ {target_lang_corpus} ------\n')
        lang = target_lang_corpus[-2:]
        for size in TARGET_CORPUS_SIZES:
            bash_script.write(f'# ---- {size} ----\n')
            for seed in range(1, 11):
                for test_path in LANG_TO_TEST_PATHS[lang]:
                    testset_name = test_path.split('/')[-1]
                    config = dict(CONFIG_TEMPLATE)
                    config['path_out'] = PATH_OUT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        fname='_'.join(testset_name.split('_')[:-2]),
                        seed=seed
                    )
                    config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                    config['dataset']['path'] = test_path
                    config['predictor']['model'] = MODEL_NAME
                    if size == 0:
                        config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                            english_dataset=eng_dataset,
                            seed=seed
                        )
                    else:
                        config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                            english_dataset=eng_dataset,
                            target_lang_corpus=target_lang_corpus,
                            target_corpus_size=size,
                            seed=seed
                        )
                    path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/'
                    path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                    if not os.path.exists(path_out_dir):
                        Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                    with open(path_config, 'w') as fout:
                        json.dump(config, fout, indent=4, ensure_ascii=False)
                    path_config = '/'.join(path_config.split('/')[1:])
                    cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                    bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate Eval Configs for NLI-Baseline (Non-Strat) Models

In [5]:
MODEL_NAME = 'morit/XLM-T-full-xnli'
ENGLISH_DATASETS = ['X_NLI_DEN', 'X_NLI_FEN', 'X_NLI_KEN', 'X_NLI']
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': 'hypotheses/en.json',
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
        'label_mapping': {
            'entailment': 0,
            'contradiction': 2
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcher': {
                'hypotheses_keys': [
                    'hate',
                    'this-text-is-hate-speech'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}

In [6]:
bash_script = open('../scripts/run_eval_nli_baseline.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)
for eng_dataset in ENGLISH_DATASETS:
    bash_script.write(f'# -------- {eng_dataset} --------\n')
    for target_lang_corpus in TARGET_LANG_CORPORA:
        bash_script.write(f'# ------ {target_lang_corpus} ------\n')
        lang = target_lang_corpus[-2:]
        for size in TARGET_CORPUS_SIZES:
            # if size != 0 and eng_dataset == 'X_NLI':
            #     continue
            bash_script.write(f'# ---- {size} ----\n')
            for seed in range(1, 11):
                for test_path in LANG_TO_TEST_PATHS[lang]:
                    testset_name = test_path.split('/')[-1]
                    config = dict(CONFIG_TEMPLATE)
                    config['path_out'] = PATH_OUT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        fname='_'.join(testset_name.split('_')[:-2]),
                        seed=seed
                    )
                    config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                    config['dataset']['path'] = test_path
                    config['predictor']['model'] = MODEL_NAME
                    if eng_dataset == 'X_NLI' and size == 0:
                        config['predictor']['checkpoint'] = None
                    elif size == 0:
                        config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                            english_dataset=eng_dataset,
                            seed=seed
                        )
                    else:
                        config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                            english_dataset=eng_dataset,
                            target_lang_corpus=target_lang_corpus,
                            target_corpus_size=size,
                            seed=seed
                        )
                    path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/baseline'
                    path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                    if not os.path.exists(path_out_dir):
                        Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                    with open(path_config, 'w') as fout:
                        json.dump(config, fout, indent=4, ensure_ascii=False)
                    path_config = '/'.join(path_config.split('/')[1:])
                    cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                    bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate Eval Configs for NLI-FBT_tc_FC_FRS Models

In [3]:
MODEL_NAME = 'morit/XLM-T-full-xnli'
ENGLISH_DATASETS = ['X_NLI_DEN', 'X_NLI_FEN', 'X_NLI_KEN', 'X_NLI']
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/FBT_tc_FC_FRS/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': 'hypotheses/en.json',
    'dataset': {
        'name': None,
        'path': None
    },
    'predictors': {
        'main': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        },
        'aux': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
        'HSCatcher': {
            'hypotheses_keys': [
            'hate',
            'that-contains-hate-speech'
            ],
            'threshold': 0.5
        }
        },
        'filters': {
            'TargetFilter': {
                'hypotheses_keys': [
                    'target_characteristics'
                ],
                'threshold': 0.1
            },
            'RecSlurFilter': {
                'hypotheses_keys': ['rec_slur'],
                'thresholds': {'about_others': 0.5, 'neg_senti': 0.5}
            },
            'CSFilter': {
                'hypotheses_keys': [
                    'stance',
                    'this-text-supports-[X]'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'max_catch_min_filter'
    }
}

In [4]:
bash_script = open('../scripts/run_eval_nli_FBT_tc_FC_FRS.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)
for eng_dataset in ENGLISH_DATASETS:
    bash_script.write(f'# -------- {eng_dataset} --------\n')
    for target_lang_corpus in TARGET_LANG_CORPORA:
        bash_script.write(f'# ------ {target_lang_corpus} ------\n')
        lang = target_lang_corpus[-2:]
        for size in TARGET_CORPUS_SIZES:
            bash_script.write(f'# ---- {size} ----\n')
            for seed in range(1, 11):
                for test_path in LANG_TO_TEST_PATHS[lang]:
                    testset_name = test_path.split('/')[-1]
                    config = dict(CONFIG_TEMPLATE)
                    config['path_out'] = PATH_OUT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        fname='_'.join(testset_name.split('_')[:-2]),
                        seed=seed
                    )
                    config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                    config['dataset']['path'] = test_path
                    config['predictors']['main']['model'] = MODEL_NAME
                    if eng_dataset == 'X_NLI' and size == 0:
                        config['predictors']['main']['checkpoint'] = None
                    elif size == 0:
                        config['predictors']['main']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                            english_dataset=eng_dataset,
                            seed=seed
                        )
                    else:
                        config['predictors']['main']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                            english_dataset=eng_dataset,
                            target_lang_corpus=target_lang_corpus,
                            target_corpus_size=size,
                            seed=seed
                        )
                    config['predictors']['aux']['model'] = MODEL_NAME
                    config['predictors']['aux']['checkpoint'] = None
                    
                    path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/FBT_tc_FC_FRS/'
                    path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                    if not os.path.exists(path_out_dir):
                        Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                    with open(path_config, 'w') as fout:
                        json.dump(config, fout, indent=4, ensure_ascii=False)
                    path_config = '/'.join(path_config.split('/')[1:])
                    cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                    bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate X_mNLI Baseline Configs

In [22]:
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
        'label_mapping': {
            'entailment': 0,
            'contradiction': 2
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcher': {
                'hypotheses_keys': [
                    'hate',
                    'this-text-is-hate-speech'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}
target_lang_corpora = ['BAS19_ES', 'HAS21_HI', 'OUS19_AR']
paths_hypotheses = ['hypotheses/translations/es.json', 'hypotheses/translations/hi.json', 'hypotheses/translations/ar.json']
model_names = ['morit/spanish_xlm_xnli', 'morit/hindi_xlm_xnli', 'morit/arabic_xlm_xnli']
eng_dataset = 'X_mNLI'

In [23]:
bash_script = open('../scripts/run_eval_mNLI_baseline.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)

for target_lang_corpus, path_hypotheses, model_name in zip(target_lang_corpora, paths_hypotheses, model_names):
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    english_dataset=eng_dataset,
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['path_hypotheses'] = path_hypotheses
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                config['predictor']['model'] = model_name
                if size == 0:
                    config['predictor']['checkpoint'] = None
                else:
                    config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/baseline'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate X_mNLI Strategy Configs

In [24]:
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/FBT_tc_FC_FRS/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/RUN{seed}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictors': {
        'main': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        },
        'aux': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
        'HSCatcher': {
            'hypotheses_keys': [
            'hate',
            'that-contains-hate-speech'
            ],
            'threshold': 0.5
        }
        },
        'filters': {
            'TargetFilter': {
                'hypotheses_keys': [
                    'target_characteristics'
                ],
                'threshold': 0.1
            },
            'RecSlurFilter': {
                'hypotheses_keys': ['rec_slur'],
                'thresholds': {'about_others': 0.5, 'neg_senti': 0.5}
            },
            'CSFilter': {
                'hypotheses_keys': [
                    'stance',
                    'this-text-supports-[X]'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'max_catch_min_filter'
    }
}
target_lang_corpora = ['BAS19_ES', 'HAS21_HI', 'OUS19_AR']
paths_hypotheses = ['hypotheses/translations/es.json', 'hypotheses/translations/hi.json', 'hypotheses/translations/ar.json']
model_names = ['morit/spanish_xlm_xnli', 'morit/hindi_xlm_xnli', 'morit/arabic_xlm_xnli']
eng_dataset = 'X_mNLI'

In [25]:
bash_script = open('../scripts/run_eval_mNLI_FBT_tc_FC_FRS.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)

for target_lang_corpus, path_hypotheses, model_name in zip(target_lang_corpora, paths_hypotheses, model_names):
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    english_dataset=eng_dataset,
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['path_hypotheses'] = path_hypotheses
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                # main
                config['predictors']['main']['model'] = model_name
                if size == 0:
                    config['predictors']['main']['checkpoint'] = None
                else:
                    config['predictors']['main']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                # aux
                config['predictors']['aux']['model'] = model_name
                config['predictors']['aux']['checkpoint'] = None
                path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/FBT_tc_FC_FRS'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate M_mNLI Baseline Configs

In [5]:
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/baseline/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/{checkpoint_name}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictor': {
        'model': None,
        'checkpoint': None,
        'label_mapping': {
            'entailment': 0,
            'contradiction': 2
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
            'HSCatcher': {
                'hypotheses_keys': [
                    'hate',
                    'this-text-is-hate-speech'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'only_HSCatcher'
    }
}
target_lang_corpora = ['BAS19_ES', 'HAS21_HI', 'OUS19_AR']
checkpoint_names = ['robertuito_xnli', 'hindi_bert_xnli', 'arabic_bert_xnli', ]
paths_hypotheses = ['hypotheses/translations/es.json', 'hypotheses/translations/hi.json', 'hypotheses/translations/ar.json']
model_names = ['pysentimiento/robertuito-base-uncased', 'neuralspace-reverie/indic-transformers-hi-bert', 'aubmindlab/bert-base-arabertv02']
eng_dataset = 'M_mNLI'

In [6]:
bash_script = open('../scripts/run_eval_M_mNLI_baseline.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)

for target_lang_corpus, path_hypotheses, model_name, checkpoint_name in zip(target_lang_corpora, paths_hypotheses, model_names, checkpoint_names):
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    english_dataset=eng_dataset,
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['path_hypotheses'] = path_hypotheses
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                config['predictor']['model'] = model_name
                if size == 0:
                    config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                        english_dataset=eng_dataset,
                        checkpoint_name=checkpoint_name
                    )
                else:
                    config['predictor']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/baseline'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()

# Generate M_mNLI Strategy Configs

In [7]:
PATH_OUT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}/FBT_tc_FC_FRS/{fname}_eval.json'
PATH_CHECKPOINT_TEMPLATE = '{english_dataset}/{target_lang_corpus}/examples_{target_corpus_size}/RUN{seed}'
PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES = '{english_dataset}/{checkpoint_name}'
CONFIG_TEMPLATE = {
    'path_out': None,
    'path_hypotheses': None,
    'dataset': {
        'name': None,
        'path': None
    },
    'predictors': {
        'main': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        },
        'aux': {
            'model': None,
            'checkpoint': None,
            'label_mapping': {
            'entailment': 0,
            'contradiction': 2
            }
        }
    },
    'prediction_pipeline': {
        'catch_threshold': 0.5,
        'catchers': {
        'HSCatcher': {
            'hypotheses_keys': [
            'hate',
            'that-contains-hate-speech'
            ],
            'threshold': 0.5
        }
        },
        'filters': {
            'TargetFilter': {
                'hypotheses_keys': [
                    'target_characteristics'
                ],
                'threshold': 0.1
            },
            'RecSlurFilter': {
                'hypotheses_keys': ['rec_slur'],
                'thresholds': {'about_others': 0.5, 'neg_senti': 0.5}
            },
            'CSFilter': {
                'hypotheses_keys': [
                    'stance',
                    'this-text-supports-[X]'
                ],
                'threshold': 0.5
            }
        },
        'comb_strat': 'max_catch_min_filter'
    }
}
target_lang_corpora = ['BAS19_ES', 'HAS21_HI', 'OUS19_AR']
checkpoint_names = ['robertuito_xnli', 'hindi_bert_xnli', 'arabic_bert_xnli', ]
paths_hypotheses = ['hypotheses/translations/es.json', 'hypotheses/translations/hi.json', 'hypotheses/translations/ar.json']
model_names = ['pysentimiento/robertuito-base-uncased', 'neuralspace-reverie/indic-transformers-hi-bert', 'aubmindlab/bert-base-arabertv02']
eng_dataset = 'M_mNLI'

In [8]:
bash_script = open('../scripts/run_eval_M_mNLI_FBT_tc_FC_FRS.sh', 'w')
bash_script.write(BASH_SCRIPT_HEADER)

for target_lang_corpus, path_hypotheses, model_name, checkpoint_name in zip(target_lang_corpora, paths_hypotheses, model_names, checkpoint_names):
    lang = target_lang_corpus[-2:]
    for size in TARGET_CORPUS_SIZES:
        bash_script.write(f'# ---- {size} ----\n')
        for seed in range(1, 11):
            for test_path in LANG_TO_TEST_PATHS[lang]:
                testset_name = test_path.split('/')[-1]
                config = dict(CONFIG_TEMPLATE)
                config['path_out'] = PATH_OUT_TEMPLATE.format(
                    english_dataset=eng_dataset,
                    target_lang_corpus=target_lang_corpus,
                    target_corpus_size=size,
                    fname='_'.join(testset_name.split('_')[:-2]),
                    seed=seed
                )
                config['path_hypotheses'] = path_hypotheses
                config['dataset']['name'] = '_'.join(testset_name.split('_')[:-2])
                config['dataset']['path'] = test_path
                # main
                config['predictors']['main']['model'] = model_name
                if size == 0:
                    config['predictors']['main']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                        english_dataset=eng_dataset,
                        checkpoint_name=checkpoint_name
                    )
                else:
                    config['predictors']['main']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE.format(
                        english_dataset=eng_dataset,
                        target_lang_corpus=target_lang_corpus,
                        target_corpus_size=size,
                        seed=seed
                    )
                # aux
                config['predictors']['aux']['model'] = model_name
                config['predictors']['aux']['checkpoint'] = PATH_CHECKPOINT_TEMPLATE_0_EXAMPLES.format(
                    english_dataset=eng_dataset,
                    checkpoint_name=checkpoint_name
                )
                path_out_dir = f'../configs/{eng_dataset}/{target_lang_corpus}/examples_{size}/RUN{seed}/FBT_tc_FC_FRS'
                path_config = os.path.join(path_out_dir, config['dataset']['name'] + '_eval.json')
                if not os.path.exists(path_out_dir):
                    Path(path_out_dir).mkdir(parents=True, exist_ok=True)
                with open(path_config, 'w') as fout:
                    json.dump(config, fout, indent=4, ensure_ascii=False)
                path_config = '/'.join(path_config.split('/')[1:])
                cmd_call = f'python3 src/evaluation.py --path_config "${{configs_dir}}/{path_config}" --gpu 0' # "${1}/
                bash_script.write(cmd_call + '\n')
bash_script.close()